In [45]:
# imports
from scisample.samplers import new_sampler
import plotly.graph_objects as go
from scipy.optimize import minimize
import numpy as np
import yaml
import pandas as pd
import tempfile
import shutil
import os

In [32]:
# functions
def rosenbrock_vector(x):
    """ Rosenbrock function 2D """
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

def rosenbrock_xy(x, y):
    """ Rosenbrock function 2D """
    return (1 - x)**2 + 100*(y - x**2)**2

def yaml_string_to_dict(yaml_string):
    yaml_data = yaml.safe_load(yaml_string)
    return yaml_data

In [33]:
# Initial guess for the minimum
x0 = np.array([0.0, 0.0])

# Use Nelder-Mead algorithm to find the minimum
result = minimize(rosenbrock_vector, x0, method='Nelder-Mead')

# Print the minimum value and corresponding x, y coordinates
print("Minimum value:", result.fun)
print("Minimum point:", result.x)

Minimum value: 3.6861769151759075e-10
Minimum point: [1.00000439 1.00001064]


In [34]:
# Contour plot
x_values = np.linspace(-1, 2, 100)  # Generate x values between -1 and 2
y_values = np.linspace(-1, 2, 100)  # Generate y values between -1 and 2
X, Y = np.meshgrid(x_values, y_values)
Z = np.log(rosenbrock_xy(X, Y))

fig = go.Figure(data=
    go.Contour(
        z=Z,
        x=x_values,
        y=y_values,
        zmin=-2,  # Set z minimum value
        zmax=6,   # Set z maximum value 
    ))
fig.show()

/var/folders/km/vxpqjv0s5ml6xgs2y6bd21yr0007gz/T/ipykernel_47356/2438218603.py:5: RuntimeWarning:

divide by zero encountered in log



extrema for new input_labels:  [-0.98826803 -0.99274791] [1.95767029 1.98832605]
down sampling to 30 best candidates from 300 total points.
samples:


,X,Y,Z
0,1.147575,0.386047,86.675626
1,-0.956210,1.949290,110.939403
2,-0.939884,-0.952057,340.646704
3,1.876379,1.827106,287.626965
4,1.947812,-0.891135,2195.920919
5,-0.386261,0.640786,26.087687
6,0.464762,1.899612,283.740197
7,0.564602,-0.952049,161.689119
8,0.080762,-0.193375,4.840893
9,0.470371,0.966573,55.831322


In [54]:
# Initial set of samples
yaml_string = """
type: best_candidate
num_samples: 30
# previous_samples: samples.csv # optional
# cost_variable: Z   # required if previous_samples is provided
# downselect_ratio: 0.3 # required if previous_samples is provided
# voxel_overlap: 1.0    # required if previous_samples is provided
parameters:
    X:
        min: -1
        max: 2
    Y:
        min: -1
        max: 2
"""
sample_dictionary = yaml_string_to_dict(yaml_string)

sampler = new_sampler(sample_dictionary)
samples = sampler.get_samples()

for sample in samples:
    sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

print("samples:")
df = pd.DataFrame(samples)
# Display the 5 smallest elements of 'Z'
smallest_values = df['Z'].nsmallest(5)

iterations = 10
print(f"5 smallest {iterations}: {smallest_values}")
while iterations > 0:
    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_file = os.path.join(temp_dir, "temp.csv")
    df.to_csv(temp_file)

    # next set of samples
    yaml_string = f"""
    type: best_candidate
    num_samples: 30
    previous_samples: {temp_file} # optional
    cost_variable: Z   # required if previous_samples is provided
    downselect_ratio: 0.3 # required if previous_samples is provided
    voxel_overlap: 1.0    # required if previous_samples is provided
    parameters:
        X:
            min: -1
            max: 2
        Y:
            min: -1
            max: 2
    """
    sample_dictionary = yaml_string_to_dict(yaml_string)

    sampler = new_sampler(sample_dictionary)
    samples = sampler.get_samples()

    for sample in samples:
        sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

    print("samples:")
    df_new = pd.DataFrame(samples)
    df = pd.concat([df, df_new], ignore_index=True)

    smallest_values = df['Z'].nsmallest(5)
    print(f"5 smallest {iterations}: {smallest_values}")

    shutil.rmtree(temp_dir)
    iterations -= 1

extrema for new input_labels:  [-0.99150813 -0.99405649] [1.9967458  1.99564053]
down sampling to 30 best candidates from 300 total points.
samples:
5 smallest 10: 23     2.336182
27     4.565914
0      5.085834
8     11.305762
10    12.583348
Name: Z, dtype: float64
extrema for new input_labels:  [-0.98442608 -0.50340789] [1.39557944 1.99961229]
down sampling to 30 best candidates from 297 total points.
samples:
5 smallest 10: 32    0.412395
45    0.857528
23    2.336182
48    2.780712
56    3.198844
Name: Z, dtype: float64
extrema for new input_labels:  [-0.99962082 -0.56857884] [1.73555772 1.78518936]
down sampling to 30 best candidates from 306 total points.
samples:
5 smallest 9: 32    0.412395
45    0.857528
71    1.921869
87    1.938642
88    1.997831
Name: Z, dtype: float64
extrema for new input_labels:  [-0.99019833 -0.45934627] [1.42279407 1.70663914]
down sampling to 30 best candidates from 297 total points.


The number of samples to keep is greater than the number of samples to generate. The number of samples to generate will be increased to the number of samples to keep.


samples:
5 smallest 8: 32     0.412395
45     0.857528
114    1.464917
119    1.773962
71     1.921869
Name: Z, dtype: float64
extrema for new input_labels:  [-0.98896072 -0.35416833] [1.58487787 1.69676481]
down sampling to 36 best candidates from 360 total points.


The number of samples to keep is greater than the number of samples to generate. The number of samples to generate will be increased to the number of samples to keep.


samples:
5 smallest 7: 137    0.081336
147    0.195562
135    0.337134
142    0.370273
32     0.412395
Name: Z, dtype: float64
extrema for new input_labels:  [-0.98710397 -0.23840342] [1.32440253 1.86890947]
down sampling to 46 best candidates from 460 total points.


...

samples:
5 smallest 4: 225    0.007872
210    0.019936
289    0.042167
182    0.067388
291    0.069509
Name: Z, dtype: float64
extrema for new input_labels:  [-0.87146745 -0.14491153] [1.38049187 1.81910786]
down sampling to 102 best candidates from 1020 total points.


The number of samples to keep is greater than the number of samples to generate. The number of samples to generate will be increased to the number of samples to keep.


samples:
5 smallest 3: 225    0.007872
210    0.019936
418    0.040808
289    0.042167
182    0.067388
Name: Z, dtype: float64
extrema for new input_labels:  [-0.81316986 -0.11575606] [1.35558441 1.84905069]
down sampling to 132 best candidates from 1320 total points.


The number of samples to keep is greater than the number of samples to generate. The number of samples to generate will be increased to the number of samples to keep.


samples:
5 smallest 2: 225    0.007872
210    0.019936
539    0.037372
507    0.039917
418    0.040808
Name: Z, dtype: float64
extrema for new input_labels:  [-0.2211837  -0.07132465] [1.33455277 1.86916781]
down sampling to 172 best candidates from 1720 total points.
samples:
5 smallest 1: 225    0.007872
700    0.013149
729    0.017844
210    0.019936
649    0.021061
Name: Z, dtype: float64


In [55]:
display(df[-20:])

,X,Y,Z
726,0.092169,0.029464,0.868124
727,0.119626,-0.046242,1.141722
728,1.112027,1.255517,0.048324
729,0.868474,0.756582,0.017844
730,1.319974,1.705067,0.241243
731,0.641003,0.504813,1.011124
732,0.195544,-0.008407,0.864720
733,-0.136302,-0.018077,1.425540
734,0.130293,-0.042478,1.109875
735,0.882947,0.660004,1.443905


In [56]:
display(df)

,X,Y,Z
0,-0.916267,0.720645,5.085834
1,1.957549,-0.975430,2312.053896
2,1.844897,1.963645,208.073484
3,0.064257,-0.985574,98.826823
4,1.018722,0.468735,32.383240
...,...,...,...
741,-0.040653,0.065660,1.492649
742,1.083782,1.234218,0.362651
743,0.045491,0.065799,1.317240
744,1.068953,1.265418,1.511683


In [58]:
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-axis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title='Scatter Plot of Column Z vs. Row Index',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()